## 과제 1
ReLu activation function과 derivative function을 구현해보세요
- Hint : np.maximum 함수 사용하면 편리합니다
- 다른 방법 사용하셔도 무방합니다


In [ ]:
import numpy as np 
import torch

#### 과제 1 코드

In [ ]:
def relu(x):

  return np.maximum(x,0)

 x = 0에서의 미분값을 어떻게 처리를 할까?

찾아보니 논문이 있네요

Numerical influence of ReLU'(0) on backpropagation:
https://arxiv.org/pdf/2106.12915.pdf

abstract만 대충 읽어보니 기본 sgd 학습에서는 ReLU'(0) = 0 효율적으로 보였다고 하네요... 오... 


In [ ]:
def d_relu(x):

 return  (x > 0).astype(int)

## 과제 2
Deep Learning Basic 코드 파일의 MLP implementation with Numpy library using MNIST dataset 코드 참고해서
Three layer MLP 일 때의 backward_pass 함수를 완성해주세요.   
- Hint : 코드 파일의 예시는 Two layer MLP


In [ ]:
import numpy as np
import sklearn.datasets

mnist = sklearn.datasets.fetch_openml('mnist_784', data_home="mnist_784")

In [ ]:
# data preprocessing

num_train = 60000
num_class = 10

x_train = np.float32(mnist.data[:num_train]).T
y_train_index = np.int32(mnist.target[:num_train]).T
x_test = np.float32(mnist.data[num_train:]).T
y_test_index = np.int32(mnist.target[num_train:]).T

# Normalization

x_train /= 255
x_test /= 255
x_size = x_train.shape[0]

y_train = np.zeros((num_class, y_train_index.shape[0]))
for idx in range(y_train_index.shape[0]):
  y_train[y_train_index[idx], idx] = 1

y_test = np.zeros((num_class, y_test_index.shape[0]))
for idx in range(y_test_index.shape[0]):
  y_test[y_test_index[idx], idx] = 1    

In [ ]:
x_train.shape

(784, 60000)

In [ ]:
np.dot([[1,2,3],[4,5,6],[7,8,9]],[1,2,3])

array([14, 32, 50])

In [ ]:
a = np.dot(params['W1'],x_train)
a.shape

(64, 60000)

In [ ]:
#parameter initialization

hidden_size1 = 64 # hidden unit size
hidden_size2 = 32
# two-layer neural network

params = {"W1": np.random.randn(hidden_size1, x_size) * np.sqrt(1/ x_size),
          "b1": np.zeros((hidden_size1, 1)) * np.sqrt(1/ x_size),
          "W2": np.random.randn(hidden_size2, hidden_size1) * np.sqrt(1/ hidden_size1),
          "b2": np.zeros((hidden_size2, 1)) * np.sqrt(1/ hidden_size1),
          'W3': np.random.randn(num_class, hidden_size2) * np.sqrt(1/ hidden_size2),
          'b3': np.zeros((num_class,1)) * np.sqrt(1/hidden_size2)
      
          }

In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

def d_sigmoid(x):
  # derivative of sigmoid
  exp = np.exp(-x)
  return (exp)/((1+exp)**2)

def softmax(x):
  exp = np.exp(x)
  return exp/np.sum(exp, axis=0)

In [ ]:
def compute_loss(y_true, y_pred):
  # loss calculation

  num_sample = y_true.shape[1]
  Li = -1 * np.sum(y_true * np.log(y_pred))
  
  return Li/num_sample

In [ ]:
def foward_pass(x, params):
  
  params["S1"] = np.dot(params["W1"], x) + params["b1"]
  params["A1"] = sigmoid(params["S1"])
  params["S2"] = np.dot(params["W2"], params["A1"]) + params["b2"]
  params["A2"] = sigmoid(params["S2"])
  params["S3"] = np.dot(params["W3"], params["A2"]) + params["b3"]
  params["A3"] = softmax(params["S3"])
 

  return params

In [ ]:
def backward_pass(x, y_true, params):

  dS3 = params["A3"] - y_true
  grads = {}

  grads["dW3"] =  np.dot(dS3, params["A2"].T)/x.shape[1]
  grads["db3"] =  (1/x.shape[1])*np.sum(dS3, axis=1, keepdims=True)/x.shape[1]

  dA2 = np.dot(params["W3"].T, dS3)
  dS2 = dA2 * d_sigmoid(params["S2"])

  grads["dW2"] = np.dot(dS2, x.T)/x.shape[1]
  grads["db2"] = np.sum(dS2, axis=1, keepdims=True)/x.shape[1]

  dA1 = np.dot(params["W2"].T, dS2)
  dS1 = dA1 * d_sigmoid(params["S1"])

  grads["dW1"] = np.dot(dS1, x.T)/x.shape[1]
  grads["db1"] = np.sum(dS1, axis=1, keepdims=True)/x.shape[1]



  return grads

## 과제 3
Deep Learning Basic 코드 파일의 MLP implementation with Pytorch library using MNIST dataset 코드 참고해서
Three layer MLP를 구한후, 학습을 돌려 보세요

hyperparameter는 다음과 같이 설정

- epochs : 100
- hiddensize : 128, 64 (two layer)
- learning_rate : 0.5

In [ ]:
from torchvision import transforms, datasets
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
trainset = datasets.MNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.MNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/train-images-idx3-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/train-labels-idx1-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/MNIST/raw



In [ ]:
batch_size = 512
train_loader = DataLoader(trainset, batch_size = batch_size, shuffle = True)

test_loader = DataLoader(testset, batch_size = batch_size, shuffle = True)

In [ ]:
trainset.data.shape

torch.Size([60000, 28, 28])

In [ ]:
trainset.targets.unique()

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
class neural_net(nn.Module):
  def __init__(self):
    super(neural_net, self).__init__()
    self.layer1 = nn.Linear(28*28, 128)
    self.layer2 = nn.Linear(128,64)
    self.layer3 = nn.Linear(64,10)
    self.relu = nn.ReLU()
  def forward(self, x):
    x = x.view(-1,28*28)
    out = self.layer1(x)
    out = self.relu(out)
    out = self.layer2(out)
    out = self.relu(out)
    out = self.layer3(out)
    return out


In [ ]:
model = neural_net()
model

neural_net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.5)
epochs = 100

In [ ]:
def train (model, train_loader,optimizer):
  model.train()

  batch_losses= []
  for data, target in train_loader:
    optimizer.zero_grad()

    output = model(data)

    loss = criterion(output, target)

    batch_losses.append(loss)
    loss.backward()

    optimizer.step()

  avg_loss = sum(batch_losses) / len(batch_losses)

  return avg_loss 



In [ ]:
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()

    batch_losses = []
    correct = 0 

    with torch.no_grad(): 
        for data, target in test_loader:
            # 예측값 생성
            output = model(data)

            # loss 계산 (이전과 동일)
            loss = criterion(output, target)
            batch_losses.append(loss)

           # Accuracy 계산
           # y pred와 y가 일치하면 correct에 1을 더해주기
            pred = output.max(1, keepdim=True)[1]

            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum().item()

    # 배치 당 평균 loss 계산 
    avg_loss =  sum(batch_losses) / len(batch_losses)

    #정확도 계산
    accuracy = 100. * correct / len(test_loader.dataset)

    return avg_loss, accuracy

In [ ]:
for epoch in range(1, epochs+1):
  train_loss = train(model, train_loader, optimizer)
  test_loss, test_accuracy = evaluate(model, test_loader)
    
  print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
         epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 0.8669	Test Loss: 0.3305	Accuracy: 89.64%
[2] Train Loss: 0.2402	Test Loss: 0.1844	Accuracy: 94.35%
[3] Train Loss: 0.1678	Test Loss: 0.1703	Accuracy: 95.05%
[4] Train Loss: 0.1330	Test Loss: 0.1344	Accuracy: 95.70%
[5] Train Loss: 0.1094	Test Loss: 0.1130	Accuracy: 96.44%
[6] Train Loss: 0.0928	Test Loss: 0.1165	Accuracy: 96.11%
[7] Train Loss: 0.0803	Test Loss: 0.0878	Accuracy: 97.14%
[8] Train Loss: 0.0694	Test Loss: 0.0982	Accuracy: 96.96%
[9] Train Loss: 0.0625	Test Loss: 0.1178	Accuracy: 96.37%
[10] Train Loss: 0.1558	Test Loss: 0.1381	Accuracy: 95.64%
[11] Train Loss: 0.0670	Test Loss: 0.0926	Accuracy: 97.13%
[12] Train Loss: 0.0532	Test Loss: 0.0853	Accuracy: 97.42%
[13] Train Loss: 0.0464	Test Loss: 0.0840	Accuracy: 97.35%
[14] Train Loss: 0.0415	Test Loss: 0.4136	Accuracy: 89.94%
[15] Train Loss: 0.1340	Test Loss: 0.0781	Accuracy: 97.57%
[16] Train Loss: 0.0446	Test Loss: 0.2564	Accuracy: 92.12%
[17] Train Loss: 0.0706	Test Loss: 0.0790	Accuracy: 97.47%
[18] T

## 과제 4
과제 3 부분의 성능을 지금까지 배운 지식을 바탕으로 향상시켜보세요

- Hint : Activation function, hyperparameter setting

1. sigmoid를 활용해보니 성능이 더 안좋아지는 것을 확인해보았고  
2. adam으로 학습해보니 학습속도가 더 느리거나 비슷하네요  
3. 추가로 dropout을 해보니 초기 학습이 비교적 빠르게 진행되지만 최종 성능은 크게 차이나지는 않네요  

4. 결론적으로 학습 방식의 문제는 아닌 것 같다는 생각이들어 모델을 좀 더 복잡하게(레이어 노드 추가) 구성해보았습니다. -> 약간의 성능 향상이 있었습니다.


In [ ]:
class neural_net2(nn.Module):
  def __init__(self):
    super(neural_net, self).__init__()
    self.layer1 = nn.Linear(28*28, 128)
    self.layer2 = nn.Linear(128,128)
    self.layer3 = nn.Linear(128,10)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    self.droppout = nn.Dropout(0.1)


  def forward(self, x):
    x = x.view(-1,28*28)
    out = self.dropout(x)
    out = self.layer1(out)
    out = self.relu(out)

    out = self.dropout(out)
    out = self.layer2(out)
    out = self.relu(out)

    out = self.layer3(out)
    return out


In [ ]:
model2 = neural_net()
model2

neural_net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr = 0.01)
epochs = 50

In [ ]:
for epoch in range(1, epochs+1):
  train_loss = train(model2, train_loader, optimizer)
  test_loss, test_accuracy = evaluate(model2, test_loader)
    
  print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
         epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 0.3352	Test Loss: 0.1393	Accuracy: 95.76%
[2] Train Loss: 0.1190	Test Loss: 0.1286	Accuracy: 96.14%
[3] Train Loss: 0.0892	Test Loss: 0.0948	Accuracy: 97.00%
[4] Train Loss: 0.0680	Test Loss: 0.0920	Accuracy: 97.22%
[5] Train Loss: 0.0572	Test Loss: 0.0871	Accuracy: 97.39%
[6] Train Loss: 0.0414	Test Loss: 0.0982	Accuracy: 97.19%
[7] Train Loss: 0.0348	Test Loss: 0.1031	Accuracy: 97.49%
[8] Train Loss: 0.0368	Test Loss: 0.0942	Accuracy: 97.58%
[9] Train Loss: 0.0276	Test Loss: 0.1224	Accuracy: 96.86%
[10] Train Loss: 0.0269	Test Loss: 0.1009	Accuracy: 97.62%
[11] Train Loss: 0.0294	Test Loss: 0.1278	Accuracy: 97.20%
[12] Train Loss: 0.0347	Test Loss: 0.1434	Accuracy: 96.93%
[13] Train Loss: 0.0285	Test Loss: 0.1220	Accuracy: 97.56%
[14] Train Loss: 0.0303	Test Loss: 0.1098	Accuracy: 97.64%
[15] Train Loss: 0.0201	Test Loss: 0.1340	Accuracy: 97.48%
[16] Train Loss: 0.0265	Test Loss: 0.1264	Accuracy: 97.58%
[17] Train Loss: 0.0191	Test Loss: 0.1279	Accuracy: 97.77%
[18] T